In [4]:
# pip install limix

In [1]:
import re
import pandas as pd
import numpy as np
import os
import seaborn as sns
import scipy.stats
import random
from matplotlib.pyplot import *
from statsmodels.nonparametric import smoothers_lowess
import statsmodels.stats.multitest
from sklearn.decomposition import PCA
from IPython.display import display, HTML
import glob

import limix
# from limix.qtl import st_scan
from numpy import dot, ones, stack
import pandas as pd


sns.set_style("ticks")
%matplotlib inline

In [2]:
geno_prefix='/hps/nobackup/hipsci/scratch/genotypes/imputed/REL-2018-01/Full_Filtered_Plink-f/hipsci.wec.gtarray.HumanCoreExome.imputed_phased.20170327.genotypes.norm.renamed.recode.filtered'
#bim,fam,bed = limix.io.read_plink(geno_prefix,verbose=False)
bim,fam,bed = limix.io.plink.read(geno_prefix,verbose=False)
bim = bim.set_index('snp')
fam = fam.set_index('iid')

In [3]:
genotypes = bed[snp_idx, sample_indices].compute()
genotypes

NameError: name 'snp_idx' is not defined

In [8]:
mydir = "/nfs/leia/research/stegle/dseaton/hipsci/singlecell_neuroseq/data/data_processed/"

In [9]:
e_df = pd.read_csv(mydir+'pool1_17_D52/pool1_17_D52.scanpy.w_metadata.w_celltype.scanpy.celltype.DA.groupedby.donor_id-celltype-treatment.mean.tsv',sep='\t', index_col=0)
e_df = pd.read_csv(mydir+'pool1_17_D52/pool1_17_D52.scanpy.w_metadata.w_celltype.scanpy.celltype.Sert.groupedby.donor_id-celltype-treatment.mean.tsv',sep='\t', index_col=0)

e_df = e_df[e_df['treatment'] == "ROT"]
sample_list = e_df.index
e_df.head(2)

,celltype,treatment,n_cells,AL627309.1,AP006222.2,RP11-206L10.3,RP11-206L10.9,LINC00115,FAM41C,RP11-54O7.3,...,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,AC011043.1,AL592183.1,AC011841.1,PNRC2-1,SRSF10-1
donor_id,,,,,,,,,,,,,,,,,,,,,
HPSI0114i-bezi_3,Sert,ROT,58,0.002768,0.130648,0.006305,0.027639,0.00000,0.009427,0.004245,...,0.930833,3.789203,2.436056,0.185731,2.906084,0.049792,0.263205,0.0,0.010365,0.080461
HPSI0114i-eipl_1,Sert,ROT,44,0.007225,0.166464,0.021807,0.000000,0.03321,0.007225,0.000000,...,0.946234,3.721917,2.385247,0.110490,3.457115,0.013738,0.000000,0.0,0.012658,0.047185


In [10]:
sample_list

Index(['HPSI0114i-bezi_3', 'HPSI0114i-eipl_1', 'HPSI0114i-fikt_3',
       'HPSI0114i-kolf_2', 'HPSI0114i-lexy_2', 'HPSI0114i-posc_1',
       'HPSI0114i-rozh_4', 'HPSI0114i-vass_1', 'HPSI0114i-wegi_1',
       'HPSI0114i-zapk_2',
       ...
       'HPSI1113i-bima_1', 'HPSI1113i-hayt_3', 'HPSI1113i-ieki_3',
       'HPSI1113i-podx_1', 'HPSI1113i-qolg_1', 'HPSI1113i-qorq_1',
       'HPSI1113i-vuud_2', 'HPSI1114i-kuul_1', 'HPSI1114i-ualf_6',
       'HPSI1213i-hehd_1'],
      dtype='object', name='donor_id', length=154)

In [75]:
# qtl_file = '/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/all_leads_5pct_FDR.csv'
# qtl_df = pd.read_csv(qtl_file,sep=',',index_col=0)
 
# qtl_df.head()

In [5]:
gene_name = 'CUL7'
snp_id = '6_43264895_C_A'

In [77]:
gene_name = 'WWC1'
snp_id = '5_167746753_C_G'

In [78]:
e_ds = e_df.loc[sample_list, gene_name]
e_ds.name = 'Expression'
e_ds.head()

donor_id
HPSI0114i-bezi_3    0.012209
HPSI0114i-eipl_1    0.022901
HPSI0114i-fikt_3    0.000000
HPSI0114i-kolf_2    0.025548
HPSI0114i-lexy_2    0.017950
Name: Expression, dtype: float64

In [79]:
e_ds.to_csv("/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/examples/WWC1_ROT_e.csv")

In [6]:
snp_idx = bim.loc[snp_id, 'i']
sample_indices = fam.loc[sample_list,'i'].tolist()

genotypes = bed[snp_idx, sample_indices].compute()

g_ds = pd.Series(genotypes, index=sample_list)
g_ds.name = 'Genotype'
g_ds.head()

NameError: name 'sample_list' is not defined

In [73]:
g_ds.to_csv("/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/examples/WWC1_g.csv")

In [37]:
df = pd.DataFrame([e_ds,g_ds]).transpose()

df.head()

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [28]:
fig = figure(figsize=(4,4))
ax = sns.boxplot(data=df, x='Genotype', y='Expression', color='.8', fliersize=False)
ax = sns.stripplot(x="Genotype", y="Expression", data=df, jitter=True, color='k')

# ylim([0.05,0.3])

NameError: name 'df' is not defined

<Figure size 288x288 with 0 Axes>